In [ ]:
import os
import warnings
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re
from transformers import pipeline
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO)  # Change to DEBUG for more details


# Suppress warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load pre-trained ResNet50 model
resnet50 = models.resnet50(pretrained=True)
resnet50.eval()
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize NLP models
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def clean_description(desc):
    non_facial_keywords = r'\b(shirt|jeans|pants|shoes|hat|scarf|jacket|coat)\b'
    clean_desc = re.sub(non_facial_keywords, '', desc, flags=re.IGNORECASE)
    return re.sub(r"\s+", " ", clean_desc).strip()

def extract_text_features(text):
    text = clean_description(text)

    feature_list = [
        'male', 'female', 'young', 'middle-aged', 'elderly',
        'blonde hair', 'brown hair', 'black hair', 'grey hair', 'white hair', 'red hair',
        'blue eyes', 'green eyes', 'brown eyes', 'hazel eyes',
        'bald', 'white', 'black', 'asian', 'hispanic', 'indian',
        'tall', 'short', 'overweight', 'slim'
    ]

    logging.info(f"Processing text: '{text}'")
    results = classifier(text, feature_list, multi_label=True)

    # Set a higher confidence threshold to avoid conflicting features
    confidence_threshold = 0.7
    attributes = {
        'sex': None,
        'age': None,
        'hair': None,
        'eyes': None,
        'height': None,
        'race': None,
        'weight': None
    }

    # Temporary dictionary to hold possible conflicting features
    temp_attributes = {
        'hair': [],
        'age': [],
        'height': []
    }

    for label, score in zip(results['labels'], results['scores']):
        if score > confidence_threshold:
            logging.debug(f"Identified feature: {label} with confidence {score}")
            if label in ['male', 'female']:
                attributes['sex'] = label
            elif label in ['young', 'middle-aged', 'elderly']:
                temp_attributes['age'].append((label, score))  # Collect age-related features
            elif 'hair' in label:
                temp_attributes['hair'].append((label.split()[0], score))  # Collect hair-related features
            elif 'eyes' in label:
                attributes['eyes'] = label.split()[0]
            elif label in ['tall', 'short']:
                temp_attributes['height'].append((label, score))  # Collect height-related features
            elif label in ['overweight', 'slim']:
                attributes['weight'] = label
            elif label in ['white', 'black', 'asian', 'hispanic', 'indian']:
                attributes['race'] = label

    # Resolve conflicts by taking the feature with the highest confidence
    if temp_attributes['hair']:
        attributes['hair'] = max(temp_attributes['hair'], key=lambda x: x[1])[0]
    if temp_attributes['age']:
        attributes['age'] = max(temp_attributes['age'], key=lambda x: x[1])[0]
    if temp_attributes['height']:
        attributes['height'] = max(temp_attributes['height'], key=lambda x: x[1])[0]

    logging.info(f"Extracted features: {attributes}")
    return attributes


def extract_image_features(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)
    with torch.no_grad():
        features = resnet50(img_tensor).squeeze().cpu().numpy()
    return features

def extract_features_for_samples(samples_folder):
    image_features = {}
    for filename in os.listdir(samples_folder):
        image_path = os.path.join(samples_folder, filename)
        if os.path.isfile(image_path):
            features = extract_image_features(image_path)
            image_features[filename] = features
    return image_features

def get_text_embedding(features):
    description = " ".join([f"{key}: {value}" for key, value in features.items() if value])
    embedding = sentence_model.encode(description)
    return embedding

def compare_embeddings(text_embedding, image_embeddings):
    # Project text_embedding to the same dimension as image_embeddings
    text_embedding_projected = np.random.rand(image_embeddings.shape[1])
    text_embedding_projected[:text_embedding.shape[0]] = text_embedding

    similarities = cosine_similarity([text_embedding_projected], image_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:3]
    return top_indices, similarities[top_indices]



def filter_person_df_with_images(person_df, samples_folder):
    """
    Filter the person DataFrame to only include entries for which
    corresponding image files exist in the samples folder.
    """
    existing_images = set(os.path.splitext(f)[0] for f in os.listdir(samples_folder) if f.endswith('.jpg'))
    return person_df[person_df['id'].astype(str).isin(existing_images)]

get_text_embedding



def match_description_to_images(description, samples_folder, person_df):

    person_df = filter_person_df_with_images(person_df, samples_folder)


    # Extract text features
    text_features = extract_text_features(description)

    # Filter person_df based on extracted features
    filtered_df = person_df.copy()

    if text_features['race']:
        filtered_df = filtered_df[filtered_df['race'].str.lower() == text_features['race'].lower()]

    # Fix: Replace 'gender' with 'sex'
    if text_features['sex']:
        filtered_df = filtered_df[filtered_df['sex'].str.lower() == text_features['sex'].lower()]

    if text_features['hair']:
        filtered_df = filtered_df[filtered_df['hair'].str.lower() == text_features['hair'].lower()]

    if text_features['eyes']:
        filtered_df = filtered_df[filtered_df['eyes'].str.lower() == text_features['eyes'].lower()]

    # Extract image features for filtered persons
    image_features = {}
    for _, row in filtered_df.iterrows():
        image_path = os.path.join(samples_folder, f"{row['id']}.jpg")
        if os.path.isfile(image_path):
            features = extract_image_features(image_path)
            image_features[row['id']] = features

    if not image_features:
        return []  # No matches found

    # Compare text features with image features
    text_embedding = get_text_embedding(text_features)
    image_embeddings = np.array(list(image_features.values()))
    image_ids = list(image_features.keys())

    top_indices, similarities = compare_embeddings(text_embedding, image_embeddings)

    # Get top 3 matches
    top_matches = []
    for idx, similarity in zip(top_indices[:3], similarities[:3]):  # Ensure we always get 3 matches if available
        image_id = image_ids[idx]
        person_info = filtered_df[filtered_df['id'].astype(str) == str(image_id)].iloc[0].to_dict()

        match_info = {
            'image_name': f"{image_id}.jpg",
            'image_path': os.path.join(samples_folder, f"{image_id}.jpg"),
            'similarity': similarity,
            'person_info': person_info
        }
        top_matches.append(match_info)

    return top_matches

def display_images(top_matches):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle("Top 3 Matched Images", fontsize=16)

    for i, match in enumerate(top_matches):
        img = Image.open(match['image_path'])
        axes[i].imshow(img)
        axes[i].set_title(f"Match {i+1}\nSimilarity: {match['similarity']:.2f}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Main function to run the process
def main():
    samples_folder = '/content/drive/MyDrive/sample_2'
    person_df = pd.read_csv('/content/drive/MyDrive/person.csv', delimiter=';')

    while True:
        description = input("Enter a description of the person you're looking for (or 'quit' to exit): ")
        if description.lower() == 'quit':
            print("Thank you for using the person search system. Goodbye!")
            break

        top_matches = match_description_to_images(description, samples_folder, person_df)

        if not top_matches:
            print("No matches found for the given description.")
        else:
            print("\nTop 3 matched images:")
            for i, match in enumerate(top_matches, 1):
                print(f"\n{i}. Image: {match['image_name']}")
                print(f"   Similarity Score: {match['similarity']:.2f}")
                print("   Person Information:")
                for key, value in match['person_info'].items():
                    print(f"   - {key.capitalize()}: {value}")

        # Display the images
        display_images(top_matches)

    print("\n")  # Add a newline for better readability

if __name__ == "__main__":
    main()

In [ ]:
import os
import warnings
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re
from transformers import pipeline
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO)  # Change to DEBUG for more details


# Suppress warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load pre-trained ResNet50 model
resnet50 = models.resnet50(pretrained=True)
resnet50.eval()
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize NLP models
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def clean_description(desc):
    non_facial_keywords = r'\b(shirt|jeans|pants|shoes|hat|scarf|jacket|coat)\b'
    clean_desc = re.sub(non_facial_keywords, '', desc, flags=re.IGNORECASE)
    return re.sub(r"\s+", " ", clean_desc).strip()

def extract_text_features(text):
    text = clean_description(text)

    feature_list = [
        'male', 'female', 'young', 'middle-aged', 'elderly',
        'blonde hair', 'brown hair', 'black hair', 'grey hair', 'white hair', 'red hair',
        'blue eyes', 'green eyes', 'brown eyes', 'hazel eyes',
        'bald', 'white', 'black', 'asian', 'hispanic', 'indian',
        'tall', 'short', 'overweight', 'slim'
    ]

    logging.info(f"Processing text: '{text}'")
    results = classifier(text, feature_list, multi_label=True)

    # Set a higher confidence threshold to avoid conflicting features
    confidence_threshold = 0.7
    attributes = {
        'sex': None,
        'age': None,
        'hair': None,
        'eyes': None,
        'height': None,
        'race': None,
        'weight': None
    }

    # Temporary dictionary to hold possible conflicting features
    temp_attributes = {
        'hair': [],
        'age': [],
        'height': []
    }

    for label, score in zip(results['labels'], results['scores']):
        if score > confidence_threshold:
            logging.debug(f"Identified feature: {label} with confidence {score}")
            if label in ['male', 'female']:
                attributes['sex'] = label
            elif label in ['young', 'middle-aged', 'elderly']:
                temp_attributes['age'].append((label, score))  # Collect age-related features
            elif 'hair' in label:
                temp_attributes['hair'].append((label.split()[0], score))  # Collect hair-related features
            elif 'eyes' in label:
                attributes['eyes'] = label.split()[0]
            elif label in ['tall', 'short']:
                temp_attributes['height'].append((label, score))  # Collect height-related features
            elif label in ['overweight', 'slim']:
                attributes['weight'] = label
            elif label in ['white', 'black', 'asian', 'hispanic', 'indian']:
                attributes['race'] = label

    # Resolve conflicts by taking the feature with the highest confidence
    if temp_attributes['hair']:
        attributes['hair'] = max(temp_attributes['hair'], key=lambda x: x[1])[0]
    if temp_attributes['age']:
        attributes['age'] = max(temp_attributes['age'], key=lambda x: x[1])[0]
    if temp_attributes['height']:
        attributes['height'] = max(temp_attributes['height'], key=lambda x: x[1])[0]

    logging.info(f"Extracted features: {attributes}")
    return attributes


def extract_image_features(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)
    with torch.no_grad():
        features = resnet50(img_tensor).squeeze().cpu().numpy()
    return features

def extract_features_for_samples(samples_folder):
    image_features = {}
    for filename in os.listdir(samples_folder):
        image_path = os.path.join(samples_folder, filename)
        if os.path.isfile(image_path):
            features = extract_image_features(image_path)
            image_features[filename] = features
    return image_features

def get_text_embedding(features):
    description = " ".join([f"{key}: {value}" for key, value in features.items() if value])
    embedding = sentence_model.encode(description)
    return embedding

def compare_embeddings(text_embedding, image_embeddings):
    # Project text_embedding to the same dimension as image_embeddings
    text_embedding_projected = np.random.rand(image_embeddings.shape[1])
    text_embedding_projected[:text_embedding.shape[0]] = text_embedding

    similarities = cosine_similarity([text_embedding_projected], image_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:3]
    return top_indices, similarities[top_indices]



def filter_person_df_with_images(person_df, samples_folder):
    """
    Filter the person DataFrame to only include entries for which
    corresponding image files exist in the samples folder.
    """
    existing_images = set(os.path.splitext(f)[0] for f in os.listdir(samples_folder) if f.endswith('.jpg'))
    return person_df[person_df['id'].astype(str).isin(existing_images)]

get_text_embedding



def match_description_to_images(description, samples_folder, person_df):

    person_df = filter_person_df_with_images(person_df, samples_folder)


    # Extract text features
    text_features = extract_text_features(description)

    # Filter person_df based on extracted features
    filtered_df = person_df.copy()

    if text_features['race']:
        filtered_df = filtered_df[filtered_df['race'].str.lower() == text_features['race'].lower()]

    # Fix: Replace 'gender' with 'sex'
    if text_features['sex']:
        filtered_df = filtered_df[filtered_df['sex'].str.lower() == text_features['sex'].lower()]

    if text_features['hair']:
        filtered_df = filtered_df[filtered_df['hair'].str.lower() == text_features['hair'].lower()]

    if text_features['eyes']:
        filtered_df = filtered_df[filtered_df['eyes'].str.lower() == text_features['eyes'].lower()]

    # Extract image features for filtered persons
    image_features = {}
    for _, row in filtered_df.iterrows():
        image_path = os.path.join(samples_folder, f"{row['id']}.jpg")
        if os.path.isfile(image_path):
            features = extract_image_features(image_path)
            image_features[row['id']] = features

    if not image_features:
        return []  # No matches found

    # Compare text features with image features
    text_embedding = get_text_embedding(text_features)
    image_embeddings = np.array(list(image_features.values()))
    image_ids = list(image_features.keys())

    top_indices, similarities = compare_embeddings(text_embedding, image_embeddings)

    # Get top 3 matches
    top_matches = []
    for idx, similarity in zip(top_indices[:3], similarities[:3]):  # Ensure we always get 3 matches if available
        image_id = image_ids[idx]
        person_info = filtered_df[filtered_df['id'].astype(str) == str(image_id)].iloc[0].to_dict()

        match_info = {
            'image_name': f"{image_id}.jpg",
            'image_path': os.path.join(samples_folder, f"{image_id}.jpg"),
            'similarity': similarity,
            'person_info': person_info
        }
        top_matches.append(match_info)

    return top_matches

def display_images(top_matches):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle("Top 3 Matched Images", fontsize=16)

    for i, match in enumerate(top_matches):
        img = Image.open(match['image_path'])
        axes[i].imshow(img)
        axes[i].set_title(f"Match {i+1}\nSimilarity: {match['similarity']:.2f}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Main function to run the process
def main():
    samples_folder = '/content/drive/MyDrive/sample_2'
    person_df = pd.read_csv('/content/drive/MyDrive/person.csv', delimiter=';')

    while True:
        description = input("Enter a description of the person you're looking for (or 'quit' to exit): ")
        if description.lower() == 'quit':
            print("Thank you for using the person search system. Goodbye!")
            break

        top_matches = match_description_to_images(description, samples_folder, person_df)

        if not top_matches:
            print("No matches found for the given description.")
        else:
            print("\nTop 3 matched images:")
            for i, match in enumerate(top_matches, 1):
                print(f"\n{i}. Image: {match['image_name']}")
                print(f"   Similarity Score: {match['similarity']:.2f}")
                print("   Person Information:")
                for key, value in match['person_info'].items():
                    print(f"   - {key.capitalize()}: {value}")

        # Display the images
        display_images(top_matches)

    print("\n")  # Add a newline for better readability

if __name__ == "__main__":
    main()
 
 # Ensure consistent case in both extracted race and DataFrame
    person_df['race'] = person_df['race'].str.lower()
    person_df['sex'] = person_df['sex'].str.lower()  # Lowercase sex for consistent matching
    Person_df['hair'] = person_df['hair'].str.lower()
    person_df['eyes'] = person_df['eyes'].str.lower()




    
    # Race mapping and filtering (case-insensitive)
    race_mapping = {
        'white': ['white', 'caucasian'],
        'black': ['black', 'african'],
        'asian': ['asian', 'east asian'],
        'hispanic': ['hispanic', 'latino'],
        'indian': ['indian', 'south asian']
    }

    if text_features['race']:
        race_matched = False
        for key, variants in race_mapping.items():
            if text_features['race'].lower() in variants:
                print(f"Filtering by race: {key}")
                filtered_df = filtered_df[filtered_df['race'] == key]
                race_matched = True
                break
        if not race_matched:
            print(f"Race '{text_features['race']}' did not match any known races.")

    # sex mapping and filtering (case-insensitive)
    sex_mapping = {
        'male': ['man', '']



In [ ]:
import os
import warnings
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re
from transformers import pipeline
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO)  # Change to DEBUG for more details


# Suppress warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load pre-trained ResNet50 model
resnet50 = models.resnet50(pretrained=True)
resnet50.eval()
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize NLP models
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def clean_description(desc):
    non_facial_keywords = r'\b(shirt|jeans|pants|shoes|hat|scarf|jacket|coat)\b'
    clean_desc = re.sub(non_facial_keywords, '', desc, flags=re.IGNORECASE)
    
    # General synonym mapping for gender, race, and other features
    synonym_mapping = {
        # Gender synonyms
        'man': 'male',
        'woman': 'female',
        'lady': 'female',
        'gentleman': 'male',
        
        # Race synonyms
        'caucasian': 'white',
        'african': 'black',
        'latino': 'hispanic',
        'east asian': 'asian',
        'south asian': 'indian',
        
        # Hair color synonyms
        'blonde': 'blonde hair',
        'brunette': 'brown hair',
        'redhead': 'red hair',

        # Comprehensive synonym mapping for gender and race

        'male': ['man', 'gentleman', 'boy'],
        'female': ['woman', 'lady', 'girl'],
        'white': ['white', 'caucasian'],
        'black': ['black', 'african'],
        'asian': ['asian', 'east asian'],
        'hispanic': ['hispanic', 'latino'],
        'indian': ['indian', 'south asian'],
        
        # Additional mappings can be added here for other features
    }

    # Replace synonyms in the description
    for standard, variations in synonym_mapping.items():
        for word in variations:
            clean_desc = re.sub(rf'\b{word}\b', standard, clean_desc, flags=re.IGNORECASE)

    return re.sub(r"\s+", " ", clean_desc).strip()
    
def extract_text_features(text):
    text = clean_description(text)

    feature_list = [
        'male', 'female', 'young', 'middle-aged', 'elderly',
        'blonde hair', 'brown hair', 'black hair', 'grey hair', 'white hair', 'red hair',
        'blue eyes', 'green eyes', 'brown eyes', 'hazel eyes',
        'bald', 'white', 'black', 'asian', 'hispanic', 'indian',
        'tall', 'short', 'overweight', 'slim'
    ]

    logging.info(f"Processing text: '{text}'")
    results = classifier(text, feature_list, multi_label=True)

    # Set a higher confidence threshold to avoid conflicting features
    confidence_threshold = 0.6
    attributes = {
        'sex': None,
        'age': None,
        'hair': None,
        'eyes': None,
        'height': None,
        'race': None,
        'weight': None
    }

    # Temporary dictionary to hold possible conflicting features
    temp_attributes = {
        'hair': [],
        'age': [],
        'height': []
    }

    for label, score in zip(results['labels'], results['scores']):
        if score > confidence_threshold:
            logging.debug(f"Identified feature: {label} with confidence {score}")
            if label in ['male', 'female', 'man', 'lady', 'woman']:
                attributes['sex'] = label
            elif label in ['young', 'middle-aged', 'elderly']:
                temp_attributes['age'].append((label, score))  # Collect age-related features
            elif 'hair' in label:
                temp_attributes['hair'].append((label.split()[0], score))  # Collect hair-related features
            elif 'eyes' in label:
                attributes['eyes'] = label.split()[0]
            elif label in ['tall', 'short']:
                temp_attributes['height'].append((label, score))  # Collect height-related features
            elif label in ['overweight', 'slim']:
                attributes['weight'] = label
            elif label in ['white', 'black', 'asian', 'hispanic', 'indian']:
                attributes['race'] = label

    # Resolve conflicts by taking the feature with the highest confidence
    if temp_attributes['hair']:
        attributes['hair'] = max(temp_attributes['hair'], key=lambda x: x[1])[0]
    if temp_attributes['age']:
        attributes['age'] = max(temp_attributes['age'], key=lambda x: x[1])[0]
    if temp_attributes['height']:
        attributes['height'] = max(temp_attributes['height'], key=lambda x: x[1])[0]

    logging.info(f"Extracted features: {attributes}")
    return attributes


def extract_image_features(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)
    with torch.no_grad():
        features = resnet50(img_tensor).squeeze().cpu().numpy()
    return features

def extract_features_for_samples(samples_folder):
    image_features = {}
    for filename in os.listdir(samples_folder):
        image_path = os.path.join(samples_folder, filename)
        if os.path.isfile(image_path):
            features = extract_image_features(image_path)
            image_features[filename] = features
    return image_features

def get_text_embedding(features):
    description = " ".join([f"{key}: {value}" for key, value in features.items() if value])
    embedding = sentence_model.encode(description)
    return embedding

def compare_embeddings(text_embedding, image_embeddings):
    # Project text_embedding to the same dimension as image_embeddings
    text_embedding_projected = np.random.rand(image_embeddings.shape[1])
    text_embedding_projected[:text_embedding.shape[0]] = text_embedding

    similarities = cosine_similarity([text_embedding_projected], image_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:3]
    return top_indices, similarities[top_indices]



def filter_person_df_with_images(person_df, samples_folder):
    """
    Filter the person DataFrame to only include entries for which
    corresponding image files exist in the samples folder.
    """
    existing_images = set(os.path.splitext(f)[0] for f in os.listdir(samples_folder) if f.endswith('.jpg'))
    return person_df[person_df['id'].astype(str).isin(existing_images)]


def match_description_to_images(description, samples_folder, person_df):
    person_df = filter_person_df_with_images(person_df, samples_folder)

    # Clean description with generalized synonym mapping
    description = clean_description(description)

    # Extract text features
    text_features = extract_text_features(description)

    # Print extracted gender and race, and available races/genders in DataFrame
    print(f"Extracted gender from description: {text_features['sex']}")
    print(f"Extracted race from description: {text_features['race']}")
    print(f"Unique races in the dataset: {person_df['race'].unique()}")
    print(f"Unique genders in the dataset: {person_df['sex'].unique()}")

 
     # Ensure consistent case in both extracted race and DataFrame
    person_df['race'] = person_df['race'].str.lower()
    person_df['sex'] = person_df['sex'].str.lower()  
    person_df['hair'] = person_df['hair'].str.lower()
    person_df['eyes'] = person_df['eyes'].str.lower()

    # Proceed with filtering
    filtered_df = person_df.copy()

    # Race mapping and filtering (case-insensitive)
    race_mapping = {
        'white': ['white', 'caucasian'],
        'black': ['black', 'african'],
        'asian': ['asian', 'east asian'],
        'hispanic': ['hispanic', 'latino'],
        'indian': ['indian', 'south asian']
    }

    if text_features['race']:
        race_matched = False
        for key, variants in race_mapping.items():
            if text_features['race'].lower() in variants:
                print(f"Filtering by race: {key}")
                filtered_df = filtered_df[filtered_df['race'] == text_features['race'].lower()]
                race_matched = True
                break
        if not race_matched:
            print(f"Race '{text_features['race']}' did not match any known races.")
            
    print(f"Filtered DataFrame for white women: \n{filtered_df[['id', 'race', 'sex']].head()}")


    # Filter by gender/sex
    if text_features['sex']:
        filtered_df = filtered_df[filtered_df['sex'].str.lower() == text_features['sex'].lower()]

    # Print the filtered DataFrame after race and gender filtering
    print(f"Filtered DataFrame after race and gender filtering:\n{filtered_df[['id', 'race', 'sex']].head()}")

    if len(filtered_df) == 0:
        print("No matches found after filtering.")
        return []


    # Filter by gender/sex
    if text_features['sex']:
        filtered_df = filtered_df[filtered_df['sex'].str.lower() == text_features['sex'].lower()]

    # Log the filtered DataFrame after gender filtering
    logging.info(f"Filtered DataFrame after gender filtering:\n{filtered_df[['id', 'race', 'sex']].head()}")
    print(f"Filtered DataFrame for white women: \n{filtered_df[['id', 'race', 'sex']].head()}")

    # Further filtering logic
    if text_features['hair']:
        filtered_df = filtered_df[filtered_df['hair'].str.lower() == text_features['hair'].lower()]

    if text_features['eyes']:
        filtered_df = filtered_df[filtered_df['eyes'].str.lower() == text_features['eyes'].lower()]

    # Log the final DataFrame after all filtering
    logging.info(f"Final DataFrame after all filtering:\n{filtered_df[['id', 'race', 'sex']].head()}")

    if len(filtered_df) == 0:
        logging.warning("No matches found after final filtering.")
        return []

    # Continue with feature matching (unchanged from previous steps)
    image_features = {}
    for _, row in filtered_df.iterrows():
        image_path = os.path.join(samples_folder, f"{row['id']}.jpg")
        if os.path.isfile(image_path):
            features = extract_image_features(image_path)
            image_features[row['id']] = features

    if not image_features:
        logging.warning("No image matches found after filtering.")
        return []

    text_embedding = get_text_embedding(text_features)
    image_embeddings = np.array(list(image_features.values()))
    image_ids = list(image_features.keys())

    top_indices, similarities = compare_embeddings(text_embedding, image_embeddings)

    # Get top 3 matches
    top_matches = []
    for idx, similarity in zip(top_indices[:3], similarities[:3]):
        image_id = image_ids[idx]
        person_info = filtered_df[filtered_df['id'].astype(str) == str(image_id)].iloc[0].to_dict()

        match_info = {
            'image_name': f"{image_id}.jpg",
            'image_path': os.path.join(samples_folder, f"{image_id}.jpg"),
            'similarity': similarity,
            'person_info': person_info
        }
        top_matches.append(match_info)

    return top_matches




def display_images(top_matches):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle("Top 3 Matched Images", fontsize=16)

    for i, match in enumerate(top_matches):
        img = Image.open(match['image_path'])
        axes[i].imshow(img)
        axes[i].set_title(f"Match {i+1}\nSimilarity: {match['similarity']:.2f}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Main function to run the process
def main():
    samples_folder = '/content/drive/MyDrive/sample_2'
    person_df = pd.read_csv('/content/drive/MyDrive/person.csv', delimiter=';')

    while True:
        description = input("Enter a description of the person you're looking for (or 'quit' to exit): ")
        if description.lower() == 'quit':
            print("Thank you for using the person search system. Goodbye!")
            break

        top_matches = match_description_to_images(description, samples_folder, person_df)

        if not top_matches:
            print("No matches found for the given description.")
        else:
            print("\nTop 3 matched images:")
            for i, match in enumerate(top_matches, 1):
                print(f"\n{i}. Image: {match['image_name']}")
                print(f"   Similarity Score: {match['similarity']:.2f}")
                print("   Person Information:")
                for key, value in match['person_info'].items():
                    print(f"   - {key.capitalize()}: {value}")

        # Display the images
        display_images(top_matches)

    print("\n")  # Add a newline for better readability

if __name__ == "__main__":
    main()